In [1]:
import cv2 as CV
import numpy as NP
import matplotlib.pyplot as PLT
import skimage.feature as F
import skimage.filters as SF
import imutils.paths as IP

In [2]:
images = list(IP.list_images('Taxis')) #...CREATE A LIST OF ALL IMAGES WE HAVE IN FOLDER named AS 'TAXIS'
sift = CV.xfeatures2d.SIFT_create() #....CREATING OBJECT 

In [3]:
A = NP.zeros((9,2))
F_X = NP.zeros((3,3))
F_Y = NP.zeros((3,3))
F_T = NP.zeros((3,3))

h_mask = NP.array([[-1,-2,-1],[0,0,0],[1,2,1]])
v_mask = NP.transpose(h_mask)

In [6]:
for k in range(0,4,2):
    PLT.figure(k+1)
    
    X1 = CV.imread(images[k])
    Y1 = X1
    I1 = CV.cvtColor(X1,CV.COLOR_BGR2GRAY)

    X2 = CV.imread(images[k+1])
    Y2 = X2
    I2 = CV.cvtColor(X2,CV.COLOR_BGR2GRAY)
    
    
    H = SF.sobel_h(I1)
    HORZ = NP.abs(H)
    V = SF.sobel_v(I1)
    VERT = NP.abs(V)

    I_X = HORZ*HORZ
    I_Y = VERT*VERT
    I_XY = HORZ * VERT

    X_B = CV.blur(I_X,(3,3))
    Y_B = CV.blur(I_Y,(3,3))
    XY_B = CV.blur(I_XY,(3,3))
    
    
    H = NP.zeros((2,2))
    R = NP.zeros((I1.shape[0],I1.shape[1]))
    for i in range(0,I1.shape[0]):
        for j in range(0,I1.shape[1]):
            H[0,0] = X_B[i,j]
            H[0,1] = XY_B[i,j]
            H[1,0] = XY_B[i,j]
            H[1,1] = Y_B[i,j]
            R[i,j] = (NP.linalg.det(H)) - ((0.04)*(NP.matrix.trace(H))**2)
    
    R_A = NP.abs(R)
    
    

    KP= sift.detectAndCompute(R_A,None)
    #X1 = CV.drawKeypoints(I1,KP,2)
    
    
    for i in range(0,len(KP)):
        X_COR = int(KP[i].pt[0])
        Y_COR = int(KP[i].pt[1])
        X_LOC = X_COR-2
        Y_LOC = Y_COR-2
    
    
        WINDOW_1 = I1[Y_LOC:Y_LOC+5,X_LOC:X_LOC+5]
        WINDOW_2 = I2[Y_LOC:Y_LOC+5,X_LOC:X_LOC+5]
        #F_X  = SF.sobel_h(WINDOW_1)
        #F_Y = SF.sobel_v(WINDOW_1)
        F_T =I1[Y_COR-1:Y_COR+2,X_COR-1:X_COR+2] - I2[Y_COR-1:Y_COR+2,X_COR-1:X_COR+2]
        
        for u in range(1,4):
            for v in range(1,4):
                W_1 = WINDOW_1[u-1:u+2,v-1:v+2]
                W_2 = WINDOW_2[u-1:u+2,v-1:v+2]
                F_X[u-1][v-1] = NP.sum(W_1*h_mask)
                F_Y[u-1][v-1] = NP.sum(W_1*v_mask)
        s1 = 0
        s2 = 0
        for p in range(0,2):
            for j in range(0,3):
                for k in range(0,3):
                    if p == 0:
                        A[s1][p] = F_X[j][k]
                        s1 = s1+1
                    else :
                        A[s2][p] = F_Y[j][k]
                        s2 = s2+1
        if NP.linalg.det(NP.dot(NP.transpose(A),A)) != 0:
            S = NP.dot(NP.dot(NP.linalg.inv(NP.dot(NP.transpose(A),A)),NP.transpose(A)),F_T.reshape(9,1))
            if int(S[1][0]) == 0 :
                S[1][0]= 0.001
        PLT.imshow(I1,'gray')
        PLT.quiver(X_COR,Y_COR,S[0][0],S[1][0],True)
                   
        PLT.axis('off')

    
#X_LIN = []
#Y_LIN = []
#X_LIN.append(X_COR)
#Y_LIN.append(Y_COR)
#PLT.quiver(X_COR,Y_COR,S[0][0],S[1][0])

#WINDOW_1 = NP.zeros((3,3))
#WINDOW_2 = NP.zeros((3,3))

error: C:\projects\opencv-python\opencv_contrib\modules\xfeatures2d\src\sift.cpp:1044: error: (-5) image is empty or has incorrect depth (!=CV_8U) in function cv::xfeatures2d::SIFT_Impl::detectAndCompute


<Figure size 432x288 with 0 Axes>